In [1]:
import pandas as pd
import sys
sys.path.append("../src")
from preprocessing import clean_text
from classifier import GPTJobDescriptionClassifier

In [2]:
df = pd.read_csv("../Engineer_20230826.csv")
df['JobDescription'] = df['JobDescription'].apply(clean_text)

In [ ]:
classifier = GPTJobDescriptionClassifier(job_postings_data=df, n_samples=1)
res = classifier.process_random_postings()

                                             sentence      category  \
0   About Lumen Lumen is guided by our belief that...     Marketing   
1   With 450,000 route fiber miles serving custome...     Marketing   
2   Learn more about Lumen's network, edge cloud, ...     Marketing   
3   The Role Manages network elements for voice an...   Description   
4             Submits capacity triggers as necessary.   Description   
5   Forecasts and trends bandwidth growth to produ...   Description   
6   Develops plans to augment capacity relief as n...   Description   
7   The Main Responsibilities + Triggers projects ...   Description   
8   + Performs engineering work and applied resear...   Description   
9           + Skilled in planning and/or engineering.   Description   
10  + Analyzes complex local and wide area network...   Description   
11  + Must be capable of handling moderately compl...   Description   
12  + Resolves difficult interoperability problems...   Description   
13    

In [7]:
print(list(res.columns))

['sentence', 'category', 'RequisitionID', 'JobTitle']


In [5]:
res.head()

,sentence,category,RequisitionID,JobTitle
0,About Lumen Lumen is guided by our belief that...,Marketing,249499,ENGINEER (all other)
1,"With 450,000 route fiber miles serving custome...",Marketing,249499,ENGINEER (all other)
2,"Learn more about Lumen's network, edge cloud, ...",Marketing,249499,ENGINEER (all other)
3,The Role Manages network elements for voice an...,Description,249499,ENGINEER (all other)
4,Submits capacity triggers as necessary.,Description,249499,ENGINEER (all other)
